In [9]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from pathlib import Path
import pandas as pd
import shap
import time
import os
import wfdb
from pathlib import Path
import re 
from collections import defaultdict
import matplotlib.pyplot as plt


class Patient:
    def __init__(self):
        self.studies = []
        self.label = []
    def __str__(self) -> str:
        return f"studies : {len(self.studies)} \n label : {len(self.label)} "

folder_path = Path('physionet.org/files/neurocritical-pediatric/1.0.0/waves')
header = [('ABP', float), ('CBFV', float)]
patient_static_map = defaultdict(Patient)


# record = wfdb.rdrecord(record_name='p007184-2116-12-15-06-58n', pn_dir='mimic3wdb-matched/1.0/p00/p007184/')
# print(record.p_signal.T)

[[   nan    nan    nan ...   109.   109.   109.]
 [   nan    nan    nan ...   109.   109.   109.]
 [   nan    nan    nan ...    nan    nan    nan]
 ...
 [   nan    nan    nan ... 63015. 63015. 63015.]
 [   nan    nan    nan ... 63062. 63062. 63062.]
 [   nan    nan    nan ...    nan    nan    nan]]


In [29]:
#LINREG ON P007184
header = [('ABP', float), ('ABP Mean', float), ('CPP', float), ('ICP Estimate', float), ('ART', float), ('UAP', float), ('UAP Mean', float), ('ICP', float)]
record = wfdb.rdsamp(record_name='p008170-2171-03-22-03-11n', pn_dir='mimic3wdb-matched/1.0/p00/p008170/', channels=[4,5,17], sampfrom=0)
for signal in record[0]:
    if signal[1] != None:
        print(signal[1])
            

0.9998494652420146
0.9998494652420146
0.9998494652420146
1.1832389400225851
1.1179221407856696
1.1681658325063737
1.9996989304840291
1.783651056085001
2.0172842225862757
1.9996989304840291
2.8664026126661777
3.13269417878591
3.984324753451847
4.283274719190038
4.833443143531749
4.933930526973158
4.315933118808495
3.9993978609680583
3.9993978609680583
3.449229436626347
2.999548395726044
2.999548395726044
2.851329505149966
2.9166463043868815
2.0499426222047332
1.9996989304840291
1.582676289202184
0.9998494652420146
0.9998494652420146
0.9998494652420146
0.9998494652420146
0.9998494652420146
0.9998494652420146
0.9998494652420146
0.9998494652420146
0.9998494652420146
0.9998494652420146
0.9998494652420146
0.9998494652420146
2.5498673548257407
5.999096791452088
5.999096791452088
5.999096791452088
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
-2.0499426222047332
-2.517208955207283
-1.6153346888206417


In [74]:
def normalize_data(data):
    data_array = data.view((data.dtype[0], len(data.dtype.names)))

    min_vals = np.min(data_array, axis=0)
    max_vals = np.max(data_array, axis=0)
    normalized_data = (data_array - min_vals) / (max_vals - min_vals)
    return normalized_data

In [75]:
def process_time_series(folder_path, header):
    for filename in folder_path.glob('*.dat'):  # Ensure we process only .dat files
        
        entity = filename.stem.split("_")
        id = entity[0][7:]        
        # Read the data record
        record = wfdb.rdrecord(str(filename.with_suffix('')), sampfrom=0)
        data = np.empty(record.sig_len, dtype=header)
        temp_labels = []
        
        for i, signal in enumerate(record.p_signal.T):
            if i == 0:
                data['ABP'] = signal
            elif i == 1:
                temp_labels = signal
            else:
                data['CBFV'] = signal

        # normalized = normalize_data(data)
        patient_static_map[id].studies.append(data)
        patient_static_map[id].label.append(temp_labels)

process_time_series(folder_path, header)
for key in patient_static_map:
    print(patient_static_map[key])

studies : 11 
 label : 11 
studies : 33 
 label : 33 
studies : 3 
 label : 3 
studies : 12 
 label : 12 
studies : 18 
 label : 18 
studies : 3 
 label : 3 
studies : 8 
 label : 8 
studies : 8 
 label : 8 
studies : 8 
 label : 8 
studies : 10 
 label : 10 
studies : 12 
 label : 12 
studies : 10 
 label : 10 


In [95]:
import numpy as np
from scipy.signal import periodogram, find_peaks
from scipy.optimize import least_squares
from scipy.optimize import minimize

import random
def estimate_icp(abp, cbfv, fs, icp):
    """
    Estimate ICP using a spectral approach from ABP and CBFV waveforms.
    
    Parameters:
    abp (numpy.array): The arterial blood pressure waveform.
    cbfv (numpy.array): The cerebral blood flow velocity waveform.
    fs (float): Sampling frequency of the waveforms.
    
    Returns:
    float: Estimated mean ICP value.
    """
    # Convert signals to numpy arrays for computation
    abp = np.array(abp)
    cbfv = np.array(cbfv)
    
    def equation(params):
        R, C = params
        q = cbfv / R
        dq_dt = np.gradient(q, 1/fs)
        return np.mean(abp - R * q - C * dq_dt)  # Example equation
    
    def objective_function( params, ground_truth):
        prediction = equation(params)
        return np.mean((prediction - ground_truth)**2)
    
    initial = [0.85,0.20]
    result = minimize(objective_function, initial, args=(icp,), method='BFGS')
    return equation(result.x), result.x


In [97]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
def rolling_window_split(data, labels, window):
    end = len(data)
    for start in range(end - window + 1):
        train_indices = list(range(start, start + window))
        yield data[train_indices], labels[train_indices]

actuals = []
estimated = []
for key in patient_static_map.keys():
    for i,study in enumerate(patient_static_map[key].studies):
        data = study 
        label = patient_static_map[key].label[i]
        for wvform, icp in rolling_window_split(data, label, 60):
            actual_icp = np.mean(icp)
            estimated_icp, params = estimate_icp(wvform['ABP'], wvform['CBFV'], 125, actual_icp)
            actuals.append(actual_icp)
            estimated.append(estimated_icp)
            print(actual_icp)
            print(estimated_icp)
            meanabp=np.mean(wvform['ABP'])
            # print(f"Mean ABP : {meanabp}, PARAMS: {params}, DIFFERENCE: {params[0] - meanabp}")
        mae = mean_absolute_error(actuals, estimated)
        rmse = np.sqrt(mean_squared_error(actuals, estimated))
        correlation = np.corrcoef(actuals, estimated)[0, 1]
        print(f"Mean Absolute Error: {mae}")
        print(f"Root Mean Square Error: {rmse}")
        print(f"Correlation Coefficient: {correlation}")
        actuals = []
        estimated = []

9.676041666666666
9.676041043778048
Mean ABP : 104.73333333333333, PARAMS: [0.43202732 1.86683808], DIFFERENCE: -104.30130601217543
9.639583333333333
9.639582006172054
Mean ABP : 104.315625, PARAMS: [0.38092918 2.09951483], DIFFERENCE: -103.93469581545193
9.604166666666666
9.604165708977144
Mean ABP : 103.91979166666667, PARAMS: [0.36761653 2.17691898], DIFFERENCE: -103.55217514119673
9.567708333333334
9.567707522456327
Mean ABP : 103.53333333333333, PARAMS: [0.39748348 2.00575796], DIFFERENCE: -103.13584985027467
9.529166666666667
9.529166476300702
Mean ABP : 103.14791666666666, PARAMS: [0.40620049 1.93208071], DIFFERENCE: -102.74171617233901
9.488541666666666
9.488540352457361
Mean ABP : 102.759375, PARAMS: [0.34185945 2.2871049 ], DIFFERENCE: -102.41751554808269
9.44375
9.443746336608626
Mean ABP : 102.36354166666666, PARAMS: [0.08077941 3.46917776], DIFFERENCE: -102.28276225331797
9.396875
9.396874125150468
Mean ABP : 101.95625, PARAMS: [ 0.43765948 -2.04219273], DIFFERENCE: -101.5

KeyboardInterrupt: 